In [ ]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import tensorflow as tf 
import matplotlib.pyplot as plt
import glob as gb
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/shanks0465/braille-character-dataset?select=Braille+Dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: chiranjeevi27
Your Kaggle Key: ··········


100%|██████████| 1.30M/1.30M [00:00<00:00, 118MB/s]

In [ ]:
train='/content/braille-character-dataset/Braille Dataset/Braille Dataset'

In [ ]:
import PIL
from PIL import Image
os.listdir(train)
for file in os.listdir(train):
    f_img = train+"/"+file
    img = Image.open(f_img)
    img = img.resize((32,32))
    img.save(f_img)

In [ ]:
from shutil import copyfile
os.mkdir('./images/')
alpha = 'a'
for i in range(0, 26): 
    os.mkdir('./images/' + alpha)
    alpha = chr(ord(alpha) + 1)

rootdir = '/content/braille-character-dataset/Braille Dataset/Braille Dataset/'
for file in os.listdir(rootdir):
    letter = file[0]
    copyfile(rootdir+file, './images/' + letter + '/' + file)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20,
                             shear_range=10,
                             validation_split=0.2,
                             samplewise_center=True,
                             samplewise_std_normalization= True,
                             width_shift_range=0.2,
                             height_shift_range=0.2
                             )
train_generator = datagen.flow_from_directory('./images/',
                                              target_size=(32,32),
                                              subset='training',
                                              shuffle=shuffle)

val_generator = datagen.flow_from_directory('./images/',
                                            target_size=(32,32),
                                            subset='validation',
                                            shuffle=shuffle)

Found 1248 images belonging to 26 classes.
Found 312 images belonging to 26 classes.


In [ ]:

pre_trained_model=tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_shape=(32,32,3)
)

24274472/24274472 [==============================] - 0s 0us/step


In [ ]:
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import layers as L
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

x = tf.keras.layers.Flatten()(pre_trained_model.output)
x=tf.keras.layers.Dense(26, activation='softmax')(x)
       

model = Model( pre_trained_model.input, x) 

print(model.summary())
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
reduce_lr = ReduceLROnPlateau(patience=15,verbose=0)
early_stop = EarlyStopping(patience=25,verbose=1)
filepath="efficient_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_10[0][0]']               
                                                                                                  
 normalization (Normalization)  (None, 32, 32, 3)    0           ['rescaling[0][0]']              
                                                                                                  
 stem_conv (Conv2D)             (None, 16, 16, 32)   864         ['normalization[0][0]']          
                                                                                            

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=666,
                              callbacks=[checkpoint,reduce_lr,early_stop],
                              verbose=1,steps_per_epoch=15)

Epoch 1/666


<ipython-input-36-b7424ec8e3b5>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


15/15 [==============================] - ETA: 0s - loss: 1.4537 - acc: 0.6500
Epoch 1: val_acc improved from -inf to 0.17628, saving model to efficient_weights.hdf5
15/15 [==============================] - 22s 669ms/step - loss: 1.4537 - acc: 0.6500 - val_loss: 3.0188 - val_acc: 0.1763 - lr: 0.0010
Epoch 2/666
15/15 [==============================] - ETA: 0s - loss: 0.5939 - acc: 0.8521
Epoch 2: val_acc did not improve from 0.17628
15/15 [==============================] - 7s 488ms/step - loss: 0.5939 - acc: 0.8521 - val_loss: 3.9476 - val_acc: 0.1058 - lr: 0.0010
Epoch 3/666
15/15 [==============================] - ETA: 0s - loss: 0.6290 - acc: 0.8167
Epoch 3: val_acc improved from 0.17628 to 0.38462, saving model to efficient_weights.hdf5
15/15 [==============================] - 7s 485ms/step - loss: 0.6290 - acc: 0.8167 - val_loss: 2.4459 - val_acc: 0.3846 - lr: 0.0010
Epoch 4/666
15/15 [==============================] - ETA: 0s - loss: 0.5859 - acc: 0.8417
Epoch 4: val_acc did not i

In [ ]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def _init_(self, w1, w2, **kwargs):
        super(WeightedAverageLayer, self)._init_(**kwargs)
        self.w1 = w1
        self.w2 = w2

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1]
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'w1': self.w1,
            'w2': self.w2,
            
        })
        return config

In [ ]:
    import numpy as np
    import tensorflow
    
    # Disable scientific notation for clarity
    np.set_printoptions(suppress=True)
    
    # Load the model
    keras_model1 = tensorflow.keras.models.load_model('efficient_weights.h5', compile=False)
    keras_model1._name = 'model1'
    keras_model2 = tensorflow.keras.models.load_model('98.h5', compile=False)
    keras_model2._name = 'model2'
    models = [keras_model1, keras_model2]
    #model_input = tf.keras.Input(shape=(125, 125, 3))
    model_input = tf.keras.Input(shape=(32, 32, 3))
    model_outputs = [model(model_input) for model in models]
    ensemble_output = 0.7 * model_outputs[0] + 0.3 * model_outputs[1]
    ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
    ensemble_model.save("Ensemble.h5")

In [ ]:
ensemble_model=tensorflow.keras.models.load_model('Ensemble.h5', compile=False,custom_objects={"WeightedAverageLayer": WeightedAverageLayer })
ensemble_model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="ensemble_weights.tf"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history=ensemble_model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=666,
                              callbacks=[checkpoint,reduce_lr,early_stop],
                              verbose=1,steps_per_epoch=15)

Epoch 1/666


<ipython-input-66-bd8b9bff2502>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=ensemble_model.fit_generator(train_generator,


15/15 [==============================] - ETA: 0s - loss: 0.5560 - acc: 0.9292
Epoch 1: val_acc improved from -inf to 0.48718, saving model to ensemble_weights.tf


15/15 [==============================] - 83s 5s/step - loss: 0.5560 - acc: 0.9292 - val_loss: 2.0570 - val_acc: 0.4872 - lr: 0.0010
Epoch 2/666
15/15 [==============================] - ETA: 0s - loss: 0.6209 - acc: 0.8813
Epoch 2: val_acc improved from 0.48718 to 0.71795, saving model to ensemble_weights.tf


15/15 [==============================] - 71s 5s/step - loss: 0.6209 - acc: 0.8813 - val_loss: 1.1589 - val_acc: 0.7179 - lr: 0.0010
Epoch 3/666
15/15 [==============================] - ETA: 0s - loss: 0.5602 - acc: 0.9125
Epoch 3: val_acc did not improve from 0.71795
15/15 [==============================] - 8s 556ms/step - loss: 0.5602 - acc: 0.9125 - val_loss: 1.6456 - val_acc: 0.5385 - lr: 0.0010
Epoch 4/666
15/15 [==============================] - ETA: 0s - loss: 0.5069 - acc: 0.9146
Epoch 4: val_acc improved from 0.71795 to 0.73397, saving model to ensemble_weights.tf


...

15/15 [==============================] - 72s 5s/step - loss: 0.3060 - acc: 0.9396 - val_loss: 0.1907 - val_acc: 0.9744 - lr: 1.0000e-04
Epoch 31/666
15/15 [==============================] - ETA: 0s - loss: 0.2310 - acc: 0.9563
Epoch 31: val_acc improved from 0.97436 to 0.97756, saving model to ensemble_weights.tf


15/15 [==============================] - 74s 5s/step - loss: 0.2310 - acc: 0.9563 - val_loss: 0.1672 - val_acc: 0.9776 - lr: 1.0000e-04
Epoch 32/666
15/15 [==============================] - ETA: 0s - loss: 0.3044 - acc: 0.9312
Epoch 32: val_acc did not improve from 0.97756
15/15 [==============================] - 9s 580ms/step - loss: 0.3044 - acc: 0.9312 - val_loss: 0.2134 - val_acc: 0.9712 - lr: 1.0000e-04
Epoch 33/666
15/15 [==============================] - ETA: 0s - loss: 0.2175 - acc: 0.9521
Epoch 33: val_acc did not improve from 0.97756
15/15 [==============================] - 9s 580ms/step - loss: 0.2175 - acc: 0.9521 - val_loss: 0.1861 - val_acc: 0.9712 - lr: 1.0000e-04
Epoch 34/666
15/15 [==============================] - ETA: 0s - loss: 0.2261 - acc: 0.9625
Epoch 34: val_acc did not improve from 0.97756
15/15 [==============================] - 8s 505ms/step - loss: 0.2261 - acc: 0.9625 - val_loss: 0.1977 - val_acc: 0.9712 - lr: 1.0000e-04
Epoch 35/666
15/15 [================

15/15 [==============================] - 74s 5s/step - loss: 0.2503 - acc: 0.9521 - val_loss: 0.1752 - val_acc: 0.9840 - lr: 1.0000e-04
Epoch 40/666
15/15 [==============================] - ETA: 0s - loss: 0.2156 - acc: 0.9604
Epoch 40: val_acc did not improve from 0.98397
15/15 [==============================] - 9s 582ms/step - loss: 0.2156 - acc: 0.9604 - val_loss: 0.2179 - val_acc: 0.9583 - lr: 1.0000e-04
Epoch 41/666
15/15 [==============================] - ETA: 0s - loss: 0.2240 - acc: 0.9542
Epoch 41: val_acc did not improve from 0.98397
15/15 [==============================] - 8s 511ms/step - loss: 0.2240 - acc: 0.9542 - val_loss: 0.2229 - val_acc: 0.9679 - lr: 1.0000e-04
Epoch 42/666
15/15 [==============================] - ETA: 0s - loss: 0.1819 - acc: 0.9688
Epoch 42: val_acc did not improve from 0.98397
15/15 [==============================] - 9s 560ms/step - loss: 0.1819 - acc: 0.9688 - val_loss: 0.2241 - val_acc: 0.9583 - lr: 1.0000e-04
Epoch 43/666
15/15 [================

15/15 [==============================] - 76s 5s/step - loss: 0.1651 - acc: 0.9750 - val_loss: 0.1550 - val_acc: 0.9872 - lr: 1.0000e-04
Epoch 54/666
15/15 [==============================] - ETA: 0s - loss: 0.1817 - acc: 0.9625
Epoch 54: val_acc did not improve from 0.98718
15/15 [==============================] - 9s 584ms/step - loss: 0.1817 - acc: 0.9625 - val_loss: 0.2142 - val_acc: 0.9583 - lr: 1.0000e-04
Epoch 55/666
15/15 [==============================] - ETA: 0s - loss: 0.2285 - acc: 0.9542
Epoch 55: val_acc did not improve from 0.98718
15/15 [==============================] - 8s 531ms/step - loss: 0.2285 - acc: 0.9542 - val_loss: 0.1782 - val_acc: 0.9744 - lr: 1.0000e-04
Epoch 56/666
15/15 [==============================] - ETA: 0s - loss: 0.2042 - acc: 0.9667
Epoch 56: val_acc did not improve from 0.98718
15/15 [==============================] - 8s 504ms/step - loss: 0.2042 - acc: 0.9667 - val_loss: 0.1900 - val_acc: 0.9744 - lr: 1.0000e-04
Epoch 57/666
15/15 [================

15/15 [==============================] - 74s 5s/step - loss: 0.1928 - acc: 0.9625 - val_loss: 0.1350 - val_acc: 0.9904 - lr: 1.0000e-05
Epoch 65/666
15/15 [==============================] - ETA: 0s - loss: 0.2247 - acc: 0.9625
Epoch 65: val_acc did not improve from 0.99038
15/15 [==============================] - 9s 580ms/step - loss: 0.2247 - acc: 0.9625 - val_loss: 0.1845 - val_acc: 0.9712 - lr: 1.0000e-05
Epoch 66/666
15/15 [==============================] - ETA: 0s - loss: 0.2295 - acc: 0.9521
Epoch 66: val_acc did not improve from 0.99038
15/15 [==============================] - 9s 584ms/step - loss: 0.2295 - acc: 0.9521 - val_loss: 0.1678 - val_acc: 0.9712 - lr: 1.0000e-05
Epoch 67/666
15/15 [==============================] - ETA: 0s - loss: 0.1902 - acc: 0.9729
Epoch 67: val_acc did not improve from 0.99038
15/15 [==============================] - 8s 521ms/step - loss: 0.1902 - acc: 0.9729 - val_loss: 0.1447 - val_acc: 0.9776 - lr: 1.0000e-05
Epoch 68/666
15/15 [================